# Batı Karadeniz (Karabük-Bartın-Zonguldak) 2025 Orman Yangınları: Etki Analizi ve İzleme Raporu

**Hazırlayan:** Yusuf Talha ARABACI
**Kurum:** Karabük Üniversitesi
**Tarih:** 08 Aralık 2025

---

## 1. Yönetici Özeti
Bu rapor, 2025 yaz sezonunda Batı Karadeniz Bölgesi'nde (Karabük, Bartın, Zonguldak) meydana gelen orman yangınlarının çevresel etkilerini, yanan alanların büyüklüğünü ve hasar derecesini bilimsel yöntemlerle ortaya koymaktadır. Analiz, **Sentinel-2** uydu görüntüleri ve **Google Earth Engine (GEE)** platformu kullanılarak Karabük Üniversitesi (KBÜ) standartlarında gerçekleştirilmiştir.

### Öne Çıkan Yöntemler
*   **Hassas Maskeleme:** Tarım arazileri ve yerleşim yerleri **ESA WorldCover 2021** verisi kullanılarak analizden çıkarılmış, sadece orman ve çalılık alanlara odaklanılmıştır. Bu sayede hasat edilmiş tarlaların "yanmış" gibi görünmesi engellenmiştir.
*   **İndeksler:** Analizde **dNDVI** (Vejetasyon Değişimi) ve **dNBR** (Yanmışlık Şiddeti Farkı) indeksleri temel alınmıştır. Karmaşık ve işlem yükü yüksek olan RBR/Severity sınıflandırmaları yerine, doğrudan fiziksel değişime odaklanan bu metrikler kullanılmıştır.


## 2. Metodoloji

Çalışmada kullanılan yöntemler uluslararası uzaktan algılama standartlarına (USGS, UN-SPIDER) uygundur.

### 2.1. Veri Kaynakları
*   **Uydu:** Sentinel-2 (MSI Sensörü) - 10m/20m Çözünürlük
*   **Referans (Öncesi) Dönem:** 01 Temmuz - 20 Temmuz 2025 (Yangın öncesi en sağlıklı bitki örtüsü)
*   **Analiz (Sonrası) Dönem:** 05 Eylül - 30 Eylül 2025 (Yangın ve duman etkisinin bittiği dönem)

### 2.2. Kullanılan İndeksler
Yangın etkisini belirlemek için aşağıdaki spektral indeksler kullanılmıştır:

1.  **dNDVI (Delta NDVI):** Bitki örtüsündeki yeşillik kaybını gösterir. Negatif değerler kayıba işaret eder.
2.  **dNBR (Delta NBR):** Yanmış alanları ve toprağın maruz kaldığı ısı stresini gösterir. Yüksek pozitif değerler şiddetli yanmayı ifade eder.

### 2.3. Sınıflandırma ve Maskeleme
*   **Arazi Örtüsü Maskesi:** **ESA WorldCover v200** verisi kullanılarak sadece **"Tree Cover" (Sınıf 10)** ve **"Shrubland" (Sınıf 20)** pikselleri analize dahil edilmiştir. **Tarım arazileri (Cropland) tamamen maskelenmiştir.**


In [1]:
# --- KURULUM ve AYARLAR ---
import os
import json
import ee
import pandas as pd
from IPython.display import IFrame, display, HTML, Image

# Proje Modülleri
from gee.pipeline import run_pipeline
from gee.utils import ee_init
from gee.aoi import get_aoi

MY_PROJECT = 'tubitak-478716'

try:
    ee_init(project=MY_PROJECT)
    print(f"✅ Google Earth Engine bağlantısı başarılı. Proje: {MY_PROJECT}")
except Exception as e:
    print(f"❌ Bağlantı Hatası: {e}")

✅ Google Earth Engine bağlantısı başarılı. Proje: tubitak-478716


In [2]:
# --- PARAMETRELER ---
pre_start = '2025-07-01'
pre_end   = '2025-07-20'
post_start = '2025-09-05'
post_end   = '2025-09-30'

base_out_dir = 'results'
print(f"📅 Analiz Dönemi: {pre_start}___{pre_end} (Öncesi) <---> {post_start}___{post_end} (Sonrası)")

📅 Analiz Dönemi: 2025-07-01___2025-07-20 (Öncesi) <---> 2025-09-05___2025-09-30 (Sonrası)


# 3. Batı Karadeniz Bölgesi Tarama (Overview)


## 3.1. Batı Karadeniz Bölgesi Tarama (Overview)
**Kapsam:** Karabük, Bartın ve Zonguldak İlleri

Bu aşamada tüm Batı Karadeniz bölgesi (Karabük, Bartın, Zonguldak) taranarak 2025 yaz sezonundaki tüm yangın kümelenmeleri (Aladağ, Ovacık, Eflani hatları) genel ölçekte gözlemlenir.


In [3]:
print("🔄 Batı Karadeniz Bölge Analizi Başlatılıyor...")

province_out_dir = os.path.join(base_out_dir, "bati_karadeniz_geneli")
province_bbox_path = "western_black_sea_bbox.geojson"

try:
    # Bölge sınırlarını al (Karabük + Bartın + Zonguldak)
    full_province_geom = get_aoi("WESTERN_BLACK_SEA")
    
    # Geçici bbox dosyası oluştur
    with open(province_bbox_path, "w", encoding="utf-8") as f:
        json.dump(full_province_geom.getInfo(), f)

    # PİPELİNE ÇALIŞTIR
    province_outputs = run_pipeline(
        pre_start=pre_start, pre_end=pre_end,
        post_start=post_start, post_end=post_end,
        aoi_geojson=province_bbox_path,
        out_dir=province_out_dir,
        project=MY_PROJECT,
        area_scale=200,     # Geniş alan için performans optimizasyonu
        min_patch_ha=5,     # 5 hektardan küçük lekeleri yoksay
        skip_severity=False,
        overlay_boundary=full_province_geom
    )
    print("✅ Bölge analizi tamamlandı.")
    
    # Sadece dNBR göster (Genel Bakış için en iyisi)
    if 'dnbr_map' in province_outputs:
       display(IFrame(src=province_outputs['dnbr_map'], width='100%', height=600))
        
except Exception as e:
    print(f"❌ HATA: {e}")
finally:
    if os.path.exists(province_bbox_path): os.remove(province_bbox_path)


🔄 Batı Karadeniz Bölge Analizi Başlatılıyor...
✅ Bölge analizi tamamlandı.
